# Protein abundance change

This notebook runs the analyses on protein abundance changes between variant and reference alleles using Cell Painting assays

In [11]:
### imports
import os
import polars as pl
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

PLATEMAP_DIR = "../../../8.2_updated_snakemake_pipeline/inputs/metadata/platemaps/{batch_id}/platemap"
PROF_DIR = "../../../8.2_updated_snakemake_pipeline/outputs/batch_profiles"
CLASS_DIR = "../../../8.2_updated_snakemake_pipeline/outputs/classification_analyses"

TRN_IMBAL_THRES = 3
MIN_CLASS_NUM = 2

## Disable truncation globally
# pl.Config.set_tbl_rows(20)  # Show all rows
# pl.Config.set_tbl_cols(40)  # Show all columns

In [10]:
# BIO_REP_BATCHES = ["2024_01_23_Batch_7", "2024_02_06_Batch_8"]
# COMBINED_BIO_REP_DIR = "2024_02_Batch_7-8"

# BIO_REP_BATCHES = ["2024_12_09_Batch_11", "2024_12_09_Batch_12"]
# COMBINED_BIO_REP_DIR = "2024_12_Batch_11-12"

# BIO_REP_BATCHES = ["2024_12_09_Batch_11_widefield", "2024_12_09_Batch_12_widefield"]
# COMBINED_BIO_REP_DIR = "2024_12_Batch_11-12_widefield"

# BIO_REP_BATCHES = ["2025_01_27_Batch_13", "2025_01_28_Batch_14"]
# COMBINED_BIO_REP_DIR = "2025_01_Batch_13-14"

BIO_REP_BATCHES = ["2025_03_17_Batch_15", "2025_03_17_Batch_16"]
COMBINED_BIO_REP_DIR = "2025_03_Batch_15-16"

OUTPUT_DIR = f"../../outputs/{COMBINED_BIO_REP_DIR}"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## 1. Get the comparable REF-VAR pairs

The cell counts between Ref. and Var. alleles should be comparable (using a TRN_IMBAL_THRES = 3).

In [12]:
# Paths
metrics_dir = "{}/{}/profiles_tcdropped_filtered_var_mad_outlier_featselect_filtcells"

metrics_df, metrics_wtvar = pl.DataFrame(), pl.DataFrame()
for batch in BIO_REP_BATCHES:
    met_dir = metrics_dir.format(CLASS_DIR, batch)
    metrics_df_batch = pl.read_csv(f"{met_dir}/metrics.csv")
    metrics_df = pl.concat([metrics_df, metrics_df_batch])
    # metrics_wtvar_batch = pl.read_csv(f"{met_dir}/metrics_summary.csv")
    # metrics_wtvar = pl.concat([metrics_wtvar, metrics_wtvar_batch])

## get individual classifiers pass the training imbalance threshold
balanced_classifiers = metrics_df.filter(
    (~pl.col("Metadata_Control"))
    & (pl.col("Training_imbalance") < TRN_IMBAL_THRES)
    & (
        (pl.col("Full_Classifier_ID").str.contains("true")) ## protein_localization detection
    )
)

balanced_class_alleles = balanced_classifiers.select(pl.col("allele_0","allele_1")).unique().to_numpy()
balanced_class_alleles = np.unique(balanced_class_alleles.flatten())
len(balanced_class_alleles)

249

## 2. CellProfiler Features

### Get the CP features for cells that passed the QC

In [13]:
pass_qc_prof_dir = "{}/{}/profiles_tcdropped_filtered_var_mad_outlier_featselect_filtcells.parquet"
cell_alleles = pl.DataFrame()

for batch_id in BIO_REP_BATCHES:
    # Get meta features
    batch_alleles = (
        pl.scan_parquet(
            pass_qc_prof_dir.format(PROF_DIR, batch_id)
        )
        # .filter(pl.col("Metadata_gene_allele").is_in(all_alleles))
        .with_columns(
            pl.concat_str(
                [
                    "Metadata_Plate",
                    "Metadata_Well",
                    "Metadata_ImageNumber",
                    "Metadata_ObjectNumber",
                ],
                separator="_",
            ).alias("Metadata_CellID")
        )
        .select([
            "Metadata_CellID",
            "Metadata_gene_allele",
            "Metadata_Well",
            "Metadata_Plate",
        ])
    )
    cell_alleles = pl.concat([cell_alleles, batch_alleles.collect()])

### Get the Cells_Intensity CP features per all cells

Merge the Cells_Intensity features to the pass-QC cells

In [14]:
combined_gfp_profiles = pl.DataFrame()
for batch_id in BIO_REP_BATCHES:
    # Get meta features
    batch_gfp_prof = (
        pl.scan_parquet(
            f"{PROF_DIR}/{batch_id}/profiles.parquet"
        ).with_columns(
            pl.concat_str(
                [
                    "Metadata_Plate",
                    "Metadata_Well",
                    "Metadata_ImageNumber",
                    "Metadata_ObjectNumber",
                ],
                separator="_",
            ).alias("Metadata_CellID")
        )
    )
    gfp_int = [i for i in batch_gfp_prof.collect_schema().names() if "Cells_Intensity" in i]
    gfp_int = ["Metadata_CellID"] + [i for i in gfp_int if "GFP" in i]

    combined_gfp_profiles = pl.concat([
        combined_gfp_profiles, 
        batch_gfp_prof.select(gfp_int).collect()
    ])

profiles = cell_alleles.join(combined_gfp_profiles, on="Metadata_CellID", how="left")

### Aggregate the cells to well profiles

In [15]:
well_profiles = (
    profiles.group_by(["Metadata_Plate", "Metadata_Well", "Metadata_gene_allele"])
    .agg(
        pl.col(col).median().alias(col)
        for col in profiles.columns
        if not col.startswith("Metadata_")
    )
    .filter(pl.col("Metadata_gene_allele").is_in(balanced_class_alleles))
    .unique()
)

well_profiles

Metadata_Plate,Metadata_Well,Metadata_gene_allele,Cells_Intensity_IntegratedIntensityEdge_GFP,Cells_Intensity_IntegratedIntensity_GFP,Cells_Intensity_LowerQuartileIntensity_GFP,Cells_Intensity_MADIntensity_GFP,Cells_Intensity_MassDisplacement_GFP,Cells_Intensity_MaxIntensityEdge_GFP,Cells_Intensity_MaxIntensity_GFP,Cells_Intensity_MeanIntensityEdge_GFP,Cells_Intensity_MeanIntensity_GFP,Cells_Intensity_MedianIntensity_GFP,Cells_Intensity_MinIntensityEdge_GFP,Cells_Intensity_MinIntensity_GFP,Cells_Intensity_StdIntensityEdge_GFP,Cells_Intensity_StdIntensity_GFP,Cells_Intensity_UpperQuartileIntensity_GFP
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2025-03-18_B16A1A2_P1T2""","""E01""","""AP2S1_Arg15His""",1.367171,45.21366,0.004318,0.001854,3.816762,0.011809,0.01801,0.004415,0.006971,0.006317,0.001611,0.001569,0.002316,0.003074,0.009053
"""2025-03-18_B16A1A2_P1T4""","""M03""","""BAG3""",2.436565,100.613016,0.009334,0.004939,3.798662,0.032706,0.052522,0.007749,0.016667,0.014781,0.001707,0.001694,0.007502,0.008472,0.021114
"""2025-03-18_B16A1A2_P1T2""","""M20""","""LITAF_Asn30Tyr""",0.778432,18.928242,0.002317,0.000438,2.722121,0.008269,0.025717,0.002539,0.003354,0.002728,0.001501,0.001449,0.000875,0.002012,0.00338
"""2025-03-18_B16A1A2_P1T2""","""D03""","""NR0B1_Trp105Cys""",0.718962,25.092147,0.002195,0.000848,3.872764,0.006823,0.03076,0.002182,0.004361,0.002912,0.001344,0.001323,0.000872,0.003267,0.004772
"""2025-03-17_B15A1A2_P1T4""","""B21""","""SMAD3_Thr261Ile""",1.128176,46.336665,0.004196,0.001858,4.057645,0.010045,0.025851,0.003603,0.007982,0.006201,0.001452,0.001432,0.001907,0.005432,0.00872
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-03-18_B16A1A2_P1T4""","""G20""","""LITAF_Pro34Thr""",0.866436,23.449473,0.002433,0.000495,3.2405,0.009297,0.033252,0.002695,0.003753,0.002893,0.00146,0.001424,0.001188,0.002577,0.003686
"""2025-03-17_B15A1A2_P1T1""","""G04""","""IMPDH1_Val317Ile""",0.473601,10.301702,0.001583,0.000093,0.382814,0.002042,0.002656,0.001601,0.001693,0.001658,0.001313,0.001229,0.000109,0.000157,0.001791
"""2025-03-17_B15A1A2_P1T4""","""F01""","""NR0B1""",0.631039,23.516238,0.001944,0.000632,3.720977,0.004491,0.021722,0.001796,0.003396,0.002536,0.001264,0.001238,0.00051,0.002094,0.00367


## 3. Calculate abundance hits

Use paired t-test to call abundance hits

In [16]:
from scipy.stats import shapiro
import re
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel
import pandas as pd


# Convert letter rows to numbers
def well_to_coordinates(well):
    row_letter, col_number = re.match(r"([A-P])(\d{2})", well).groups()
    row_index = ord(row_letter) - ord('A') + 1  # Convert 'A'->1, 'B'->2, ..., 'P'->16
    col_index = int(col_number)  # Convert string column to integer
    return well, row_index, col_index


# Compute distances from edges and find the most centered well
def compute_distance(row, col):
    return min(row - 1, 16 - row, col - 1, 24 - col)  # Distance from nearest edge


## Abundance recalculation test: by Cell MeanIntensity
def paired_ttest(dat, reference: str, var: str, intensity_type: str="Cells_Intensity_IntegratedIntensity_GFP"):
    # pivot to wide: one row per plate
    wide_gfp = dat.pivot(index="Metadata_Plate",
                        columns="Metadata_gene_allele",
                        values=intensity_type)
    # drop any plate that doesn’t have both measurements
    wide_gfp = wide_gfp.dropna(subset=[reference, var])
    if wide_gfp.shape[0] >= 3:
        # now run paired t-test
        t_stat, p_val = ttest_rel(wide_gfp[reference], wide_gfp[var])
    else:
        t_stat, p_val = None, None

    # ## perform stat test
    # dat = dat.dropna().sort_values(["Metadata_Plate","Metadata_gene_allele"])
    # # Assuming well_abun_stats is a DataFrame with columns 'reference_abundance' and 'variant_abundance'
    # reference_abundance = dat[dat["Metadata_gene_allele"]==reference][intensity_type].values
    # variant_abundance = dat[dat["Metadata_gene_allele"]==var][intensity_type].values
    # t_stat, p_val = ttest_rel(variant_abundance, reference_abundance)
    
    # Calculate Cohen's d
    mean_diff = np.mean(wide_gfp[var]) - np.mean(wide_gfp[reference])
    pooled_std = np.sqrt((np.std(wide_gfp[var], ddof=1) ** 2 + np.std(wide_gfp[reference], ddof=1) ** 2) / 2)
    cohen_d = mean_diff / pooled_std

    summary_df = pl.DataFrame(
        {
            "t_stat": t_stat,
            "p_val": p_val,
            "cohen_d": cohen_d
        }
    )
    summary_df = summary_df.with_columns(
        pl.lit(reference).alias("Gene"), pl.lit(var).alias("Variant")
    )
    return summary_df

In [17]:
well_abun_stats = []
for allele in tqdm(well_profiles.select(pl.col("Metadata_gene_allele")).to_pandas()["Metadata_gene_allele"].unique()):
    if allele is None or allele.split("_")[0] == allele:
        continue

    reference = allele.split("_")[0]
    temp_prof = well_profiles.filter(
        (pl.col("Metadata_gene_allele") == allele) | (pl.col("Metadata_gene_allele") == reference)
    ).to_pandas()
    
    if (temp_prof["Metadata_gene_allele"].unique().shape[0] < 2):
        # print(temp_prof)
        continue

    var_profiles = temp_prof[temp_prof["Metadata_gene_allele"]==allele]
    ref_profiles = temp_prof[(temp_prof["Metadata_gene_allele"]==reference)&(temp_prof["Metadata_Plate"].isin(var_profiles["Metadata_Plate"].unique()))]
    temp_prof = pd.concat([var_profiles, ref_profiles])

    ref_wells = ref_profiles["Metadata_Well"].unique()
    var_wells = var_profiles["Metadata_Well"].unique()
    ref_var_pairs = [(ref_well, var_well) for ref_well in ref_wells for var_well in var_wells]
    
    ## Per each ref-var well pair on the SAME plate, train and test the classifier
    for ref_var in ref_var_pairs:
        ## sort the wells to make sure they are from the same plate
        df_sampled = temp_prof[temp_prof["Metadata_Well"].isin(ref_var)].dropna().sort_values(["Metadata_Plate","Metadata_gene_allele"])
        paired_t_res = paired_ttest(
            dat=df_sampled,
            reference=reference,
            var=allele
        ).with_columns(
            pl.lit(ref_var[0]).alias("Ref_well"),
            pl.lit(ref_var[1]).alias("Var_well")
        )
        well_abun_stats.append(
            paired_t_res
        )

well_abun_stats = pl.concat(well_abun_stats, how="vertical")
well_abun_stats = well_abun_stats.rename({"t_stat": "U2OS_t"})
well_abun_stats = well_abun_stats.sort(["Gene", "Variant", "U2OS_t", "p_val", "cohen_d"])
well_abun_stats

100%|██████████| 249/249 [00:01<00:00, 193.44it/s]


U2OS_t,p_val,cohen_d,Gene,Variant,Ref_well,Var_well
f64,f64,f64,str,str,str,str
-6.709696,0.000275,2.715262,"""AP2S1""","""AP2S1_Arg15Cys""","""A01""","""C01"""
-7.557733,0.004804,5.181297,"""AP2S1""","""AP2S1_Arg15His""","""A01""","""E01"""
-10.417362,0.000016,4.139008,"""APOA1""","""APOA1_Ala188Ser""","""G01""","""K03"""
-7.47105,0.000141,3.622423,"""APOA1""","""APOA1_Ala199Pro""","""G01""","""C03"""
-5.271115,0.001159,1.79803,"""APOA1""","""APOA1_Arg34Leu""","""G01""","""I01"""
…,…,…,…,…,…,…
6.426477,0.000358,-1.110552,"""TPM1""","""TPM1_Ser215Leu""","""L04""","""J04"""
-25.27152,0.000015,15.349455,"""TPM1""","""TPM1_Val95Ala""","""P06""","""F08"""
0.269709,0.795162,-0.118346,"""TPM1""","""TPM1_Val95Ala""","""P23""","""F08"""


In [18]:
well_abun_stats.write_csv(f"../../outputs/{COMBINED_BIO_REP_DIR}/well-level_prot-abundance_changes.csv")

### 3.1 Single-plate layout

In [8]:
well_abun_stats = []

for allele in tqdm(well_profiles.select(pl.col("Metadata_gene_allele")).to_pandas()["Metadata_gene_allele"].unique()):
    if allele is None or allele.split("_")[0] == allele:
        continue

    reference = allele.split("_")[0]
    temp_prof = well_profiles.filter(
        (pl.col("Metadata_gene_allele") == allele) | (pl.col("Metadata_gene_allele") == reference)
    ).to_pandas()
    if (temp_prof["Metadata_gene_allele"].unique().shape[0] < 2):
        # print(temp_prof)
        continue

    var_profiles = temp_prof[temp_prof["Metadata_gene_allele"]==allele]
    ref_profiles = temp_prof[(temp_prof["Metadata_gene_allele"]==reference)&(temp_prof["Metadata_Plate"].isin(var_profiles["Metadata_Plate"].unique()))]
    temp_prof = pd.concat([var_profiles, ref_profiles])
    
    ## Per each ref-var well pair on the SAME plate, train and test the classifier
    ## sort the wells to make sure they are from the same plate
    df_sampled = pd.DataFrame()
    for plate in temp_prof["Metadata_Plate"].unique():
        dat = temp_prof[temp_prof["Metadata_Plate"]==plate].dropna().sort_values(["Metadata_gene_allele"])
        # count rows per group
        group_counts = dat.groupby("Metadata_gene_allele").size()  #  [oai_citation:0‡Pandas](https://pandas.pydata.org/docs/user_guide/groupby.html?utm_source=chatgpt.com)
        min_count = group_counts.min()
        # print("Minimum rows in any group:", min_count)
        
        shuffled = dat.sample(frac=1, random_state=42).reset_index(drop=True)  #  [oai_citation:2‡Vultr Docs](https://docs.vultr.com/python/third-party/pandas/DataFrame/sample?utm_source=chatgpt.com)
        # Then take the first min_count rows per group
        sampled_df2 = (
            shuffled
            .groupby("Metadata_gene_allele", group_keys=False)  #  [oai_citation:3‡Built In](https://builtin.com/data-science/pandas-groupby?utm_source=chatgpt.com)
            .head(min_count)
        )
        df_sampled = pd.concat([df_sampled, sampled_df2])
    
    # print(df_sampled)
    if df_sampled.shape[0] > 3:
        # now run paired t-test
        t_stat, p_val = ttest_rel(df_sampled.loc[df_sampled["Metadata_gene_allele"]==reference, "Cells_Intensity_IntegratedIntensity_GFP"].values, 
                                  df_sampled.loc[df_sampled["Metadata_gene_allele"]==allele, "Cells_Intensity_IntegratedIntensity_GFP"].values)
        mean_diff = np.mean(df_sampled.loc[df_sampled["Metadata_gene_allele"]==allele, "Cells_Intensity_IntegratedIntensity_GFP"].values) - \
            np.mean(df_sampled.loc[df_sampled["Metadata_gene_allele"]==reference, "Cells_Intensity_IntegratedIntensity_GFP"].values)
        pooled_std = np.sqrt((np.std(df_sampled.loc[df_sampled["Metadata_gene_allele"]==allele, "Cells_Intensity_IntegratedIntensity_GFP"].values, ddof=1) ** 2 + \
                              np.std(df_sampled.loc[df_sampled["Metadata_gene_allele"]==reference, "Cells_Intensity_IntegratedIntensity_GFP"].values, ddof=1) ** 2) / 2)
        cohen_d = mean_diff / pooled_std
    else:
        t_stat, p_val, cohen_d  = None, None, None
        
    # break
    summary_df = pl.DataFrame(
        {
            "t_stat": t_stat,
            "p_val": p_val,
            "cohen_d": cohen_d
        }
    )
    summary_df = summary_df.with_columns(
        pl.lit(reference).alias("Gene"), pl.lit(allele).alias("Variant")
    )
    # print(paired_t_res)
    well_abun_stats.append(
        summary_df
    )

well_abun_stats = pl.concat(well_abun_stats,  how="vertical")
well_abun_stats = well_abun_stats.rename({"t_stat": "U2OS_t"})
well_abun_stats = well_abun_stats.sort(["Gene", "Variant", "U2OS_t", "p_val", "cohen_d"])
display(well_abun_stats)
well_abun_stats.write_csv(f"../../outputs/{COMBINED_BIO_REP_DIR}/well-level_prot-abundance_changes.csv")

100%|██████████| 77/77 [00:00<00:00, 159.85it/s]


U2OS_t,p_val,cohen_d,Gene,Variant
f64,f64,f64,str,str
15.103341,0.000001,-8.027915,"""AGXT""","""AGXT_Asp201Asn"""
0.941877,0.377616,-0.400286,"""GSS""","""GSS_Arg125Cys"""
2.83516,0.025219,-1.581071,"""HPRT1""","""HPRT1_His204Asp"""
-4.215618,0.003958,2.13442,"""MLH1""","""MLH1_Ala160Val"""
-5.645818,0.000778,3.137799,"""MLH1""","""MLH1_Ala29Gly"""
…,…,…,…,…
-5.384443,0.001025,2.394275,"""STXBP1""","""STXBP1_Pro480Leu"""
-9.601458,0.000028,5.169416,"""STXBP1""","""STXBP1_Pro94Leu"""
-10.918077,0.000012,6.403128,"""STXBP1""","""STXBP1_Thr419Met"""
